In [1]:
from csgo.parser import DemoParser
import numpy
import csv
import pandas
import pyserialem
import os
from tabulate import tabulate
from sklearn import preprocessing

In [2]:
def getDemoPaths(directory):
    demoPaths = []
    for filename in os.listdir(directory):
        demoPaths.append(os.path.join(directory, filename))
    return demoPaths

In [3]:
def generateDemoParsers(paths):
    demo_parsers = []
    for x in paths:
        demoParser = DemoParser(demofile=x, demo_id = x.split('/')[-1].split('.')[0].split('_')[0], parse_rate=128)
        demo_parsers.append(demoParser)
    print("# Demo Parsers: " + str(len(demo_parsers)))
    return demo_parsers

In [4]:
def generateData(parsers):
    killData, roundData, damageData = [], [], [],
    for x in parsers:
        x.parse()
        killData.append(x._parse_kills(return_type="df"))
        roundData.append(x._parse_rounds(return_type="df"))
        damageData.append(x._parse_damages(return_type="df"))
    return (killData, roundData, damageData)

In [5]:
def createSidedCSVDFWithScaledPer30(players, damageData, killData, roundData, teams):
    locationNames = []
    playerNames = []

    Kills = []
    killsPer30 = []
    Deaths = []
    deathsPer30 = []
    KdDifferential = []
    kdPer30 = []
    DamageDealt = []
    damageDealtPer30 = []
    DamageTaken = []
    damageTakenPer30 = []
    DamageDifferential = []
    damageDiffPer30 = []
    sides = []
    totalRounds = []
    dataframe = {}
    for x in players.keys():
        for y in killData[killData['AttackerName'].isin(players[x]) | killData['VictimName'].isin(players[x])].VictimAreaName.value_counts().index:
            tempCTKills = len(killData[killData['AttackerName'].isin(players[x])][killData['AttackerAreaName'] == y][killData['AttackerSide'] == 'CT'])
            tempCTDeaths = len(killData[killData['VictimName'].isin(players[x])][killData['VictimAreaName'] == y][killData['VictimSide'] == 'CT'])
            tempCTKdDifferential = tempCTKills - tempCTDeaths
            
            tempCTDamageDealt = damageData[damageData['AttackerName'].isin(players[x])][damageData['AttackerAreaName'] == y][damageData['AttackerSide'] == 'CT'].HpDamageTaken.sum()
            tempCTDamageTaken = damageData[damageData['VictimName'].isin(players[x])][damageData['VictimAreaName'] == y][damageData['VictimSide'] == 'CT'].HpDamageTaken.sum()
            tempCTDamageDifferential = tempCTDamageDealt - tempCTDamageTaken
                                                                                                             
            tempTKills = len(killData[killData['AttackerName'].isin(players[x])][killData['AttackerAreaName'] == y][killData['AttackerSide'] == 'T'])
            tempTDeaths = len(killData[killData['VictimName'].isin(players[x])][killData['VictimAreaName'] == y][killData['VictimSide'] == 'T'])
            tempTKdDifferential = tempTKills - tempTDeaths
                                                                                                             
            tempTDamageDealt = damageData[damageData['AttackerName'].isin(players[x])][damageData['AttackerAreaName'] == y][damageData['AttackerSide'] == 'T'].HpDamageTaken.sum()
            tempTDamageTaken = damageData[damageData['VictimName'].isin(players[x])][damageData['VictimAreaName'] == y][damageData['VictimSide'] == 'T'].HpDamageTaken.sum()
            tempTDamageDifferential = tempTDamageDealt - tempTDamageTaken
            
            
            locationNames.append(y)
            locationNames.append(y)
            
            playerNames.append(x)
            playerNames.append(x)
            
            Kills.append(tempCTKills)
            Kills.append(tempTKills)
            
            
            Deaths.append(tempCTDeaths)
            Deaths.append(tempTDeaths)
            
            
            KdDifferential.append(tempCTKdDifferential)
            KdDifferential.append(tempTKdDifferential)
            
            
            DamageDealt.append(tempCTDamageDealt)
            DamageTaken.append(tempCTDamageTaken)
            DamageDifferential.append(tempCTDamageDifferential)
            
            DamageDealt.append(tempTDamageDealt)
            DamageTaken.append(tempTDamageTaken)
            DamageDifferential.append(tempTDamageDifferential)
            
            sides.append('CT')
            sides.append('T')
            
            for y in teams.keys():
                if x in teams[y]['Players']:
                    totalRounds.append(teams[y]['CT'])
                    totalRounds.append(teams[y]['T'])
                    break
            
            damageDiffPer30.append(tempCTDamageDifferential / totalRounds[-2] * 30)
            damageDiffPer30.append(tempTDamageDifferential / totalRounds[-1] * 30)

            kdPer30.append(tempCTKdDifferential / totalRounds[-2] * 30)
            kdPer30.append(tempTKdDifferential / totalRounds[-1] * 30)
            
            
            killsPer30.append(tempCTKills / totalRounds[-2] * 30)
            killsPer30.append(tempTKills / totalRounds[-1] * 30)
            
            deathsPer30.append(tempCTDeaths / totalRounds[-2] * 30)
            deathsPer30.append(tempTDeaths / totalRounds[-1] * 30)

            damageDealtPer30.append(tempCTDamageDealt / totalRounds[-2] * 30)
            damageDealtPer30.append(tempTDamageDealt / totalRounds[-1] * 30)
            
            damageTakenPer30.append(tempCTDamageTaken / totalRounds[-2] * 30)
            damageTakenPer30.append(tempTDamageTaken / totalRounds[-1] * 30)
            
    
    dataframe['Location'] = locationNames
    dataframe['Player'] = playerNames
    dataframe['Kills'] = Kills
    dataframe['Deaths'] = Deaths
    dataframe['K/D'] = KdDifferential
    dataframe['Damage Dealt'] = DamageDealt
    dataframe['Damage Taken'] = DamageTaken
    dataframe['Damage Differential'] = DamageDifferential
    dataframe['Side'] = sides
    dataframe['Total Rounds'] = totalRounds
    dataframe['Damage Diff Per 30'] = damageDiffPer30
    dataframe['KD Diff Per 30'] = kdPer30
    dataframe['Kills Per 30'] = killsPer30
    dataframe['Deaths Per 30'] = deathsPer30
    dataframe['Damage Dealt Per 30'] = damageDealtPer30
    dataframe['Damage Taken Per 30'] = damageTakenPer30   
    
    return pandas.DataFrame.from_dict(dataframe, orient='columns')

In [6]:
def concat_data(aggregate_data):
    final_df = None
    for x in aggregate_data:
        final_df = pandas.concat([final_df, x], ignore_index=True)
    return final_df

In [7]:
demo_paths = "/mnt/d/ESL_COLOGNE"

In [8]:
demoPaths = getDemoPaths(demo_paths)

In [9]:
demos = []
for x in demoPaths:
    if "ancient" not in x:
        demos.append(x)

In [10]:
demos

['/mnt/d/ESL_COLOGNE/evil-geniuses-vs-faze-mirage.dem',
 '/mnt/d/ESL_COLOGNE/faze-vs-complexity-m1-dust2.dem',
 '/mnt/d/ESL_COLOGNE/faze-vs-complexity-m2-nuke.dem']

In [11]:
mirageMappingPath = './mappings/mappings/'
infernoMappingPath = './mappings/mappings/de_inferno_place_mappings.txt'

In [12]:
file = open(infernoMappingPath, 'r')
lines = file.readlines()
file.close()

In [13]:
for x in range(len(lines)):
    if(lines[x][-1] == '\n'):
        lines[x] = lines[x][:-1]

In [14]:
mappedPlaceNames = {}
for x in lines:
    if(x != ''):
        mappedPlaceNames[x.split(':')[0]] = x.split(':')[1]

In [15]:
mappingFile = os.path.join(mirageMappingPath, os.listdir(mirageMappingPath)[1])

In [16]:
file = open(mappingFile, 'r')
lines = file.readlines()
file.close()

In [17]:
for x in range(len(lines)):
    if(lines[x][-1] == '\n'):
        lines[x] = lines[x][:-1]

In [18]:
mappedPlaceName = {}
for x in lines:
    if(x != ''):
        mappedPlaceName[x.split(':')[0]] = x.split(':')[1]

In [19]:
parsers = generateDemoParsers(demos)

22:06:42 [INFO] Go version>=1.14.0
22:06:42 [INFO] Initialized CSGODemoParser with demofile /mnt/d/ESL_COLOGNE/evil-geniuses-vs-faze-mirage.dem
22:06:42 [INFO] Setting demo id to evil-geniuses-vs-faze-mirage
22:06:42 [INFO] Setting parse rate to 128
22:06:42 [INFO] Go version>=1.14.0
22:06:42 [INFO] Initialized CSGODemoParser with demofile /mnt/d/ESL_COLOGNE/faze-vs-complexity-m1-dust2.dem
22:06:42 [INFO] Setting demo id to faze-vs-complexity-m1-dust2
22:06:42 [INFO] Setting parse rate to 128
22:06:42 [INFO] Go version>=1.14.0
22:06:42 [INFO] Initialized CSGODemoParser with demofile /mnt/d/ESL_COLOGNE/faze-vs-complexity-m2-nuke.dem
22:06:42 [INFO] Setting demo id to faze-vs-complexity-m2-nuke
22:06:42 [INFO] Setting parse rate to 128


# Demo Parsers: 3


In [20]:
kills, rounds, damages = generateData(parsers)

22:06:47 [INFO] Running Golang parser from /home/nk095x/cs_demos/csgo/csgo/parser/
22:06:47 [INFO] Looking for file at /mnt/d/ESL_COLOGNE/evil-geniuses-vs-faze-mirage.dem
22:07:18 [INFO] Wrote demo parse output to evil-geniuses-vs-faze-mirage.json
22:07:18 [INFO] Reading in JSON from evil-geniuses-vs-faze-mirage.json
22:07:19 [INFO] JSON data loaded, available in the `json` attribute to parser
22:07:19 [INFO] Successfully parsed JSON output
22:07:19 [INFO] Successfully returned JSON output
22:07:19 [INFO] Parsed kills to Pandas DataFrame
22:07:19 [INFO] Parsed rounds to Pandas DataFrame
22:07:19 [INFO] Parsed damages to Pandas DataFrame
22:07:19 [INFO] Running Golang parser from /home/nk095x/cs_demos/csgo/csgo/parser/
22:07:19 [INFO] Looking for file at /mnt/d/ESL_COLOGNE/faze-vs-complexity-m1-dust2.dem
22:07:39 [INFO] Wrote demo parse output to faze-vs-complexity-m1-dust2.json
22:07:39 [INFO] Reading in JSON from faze-vs-complexity-m1-dust2.json
22:07:40 [INFO] JSON data loaded, avail

In [24]:
kill = concat_data(kills)
rounds_ = concat_data(rounds)
damage = concat_data(damages)

In [25]:
mirageKill = kill[kill.MapName == 'de_mirage']
infernoKill = kill[kill.MapName == 'de_inferno']
kill = kill[(kill['MapName'] != 'de_mirage') & (kill['MapName'] != 'de_inferno')]

In [26]:
mirageDamage = damage[damage.MapName == 'de_mirage']
infernoDamage = damage[damage.MapName == 'de_inferno']
damage = damage[(damage['MapName'] != 'de_mirage') & (damage['MapName'] != 'de_inferno')]

In [27]:
mirageRound = rounds_[rounds_.MapName == 'de_mirage']
infernoRound = rounds_[rounds_.MapName == 'de_inferno']
rounds_ = rounds_[(rounds_['MapName'] != 'de_mirage') & (rounds_['MapName'] != 'de_inferno')]

In [28]:
mirageKill['AttackerAreaName'].replace(mappedPlaceName, inplace=True)
mirageKill['VictimAreaName'].replace(mappedPlaceName, inplace=True)

In [29]:
mirageDamage['AttackerAreaName'].replace(mappedPlaceName, inplace=True)
mirageDamage['VictimAreaName'].replace(mappedPlaceName, inplace=True)

In [32]:
teams = {}
for x in mirageRound.WinningTeam.unique():
    teams[x] = {'CT' : 0, 'T' : 0}
print(teams.keys())
for i, r in mirageRound.iterrows():
    teams[r['WinningTeam']][r['WinningSide']] += 1
    if(r['WinningSide'] == 'CT'):
        teams[r['LosingTeam']]['T'] += 1
    else:
        teams[r['LosingTeam']]['CT'] += 1
for x in teams.keys():
    teams[x]['Players'] = []

dict_keys(['Evil Geniuses', 'FaZe Clan'])


In [34]:
for x in mirageKill.AttackerName.unique():
    if x != None and x not in teams[mirageKill[mirageKill['AttackerName'] == x].AttackerTeam.iloc[0]]['Players']:
        teams[mirageKill[mirageKill['AttackerName'] == x].AttackerTeam.iloc[0]]['Players'].append(x)

In [36]:
player_names = list(mirageKill.AttackerName.value_counts().index)
player_name = {}
for x in player_names:
    player_name[x] = [x]

In [37]:
damageDifferentials = createSidedCSVDFWithScaledPer30(player_name, mirageDamage, mirageKill, mirageRound, teams)

/home/nk095x/.local/lib/python3.6/site-packages/ipykernel_launcher.py:22: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/home/nk095x/.local/lib/python3.6/site-packages/ipykernel_launcher.py:23: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/home/nk095x/.local/lib/python3.6/site-packages/ipykernel_launcher.py:26: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/home/nk095x/.local/lib/python3.6/site-packages/ipykernel_launcher.py:27: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/home/nk095x/.local/lib/python3.6/site-packages/ipykernel_launcher.py:30: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/home/nk095x/.local/lib/python3.6/site-packages/ipykernel_launcher.py:31: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/home/nk095x/.local/lib/python3.6/site-packages/ipykernel_launcher.py:34: UserWarning: Boolean Serie

In [40]:
damageDifferentials.to_csv('./damageDifferentials/faze_mirage_playin.csv')